## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [16]:
%matplotlib inline

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
# answer below:
wine.isnull().mean()

fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

In [7]:
wine.quality.value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [12]:
# answer below
bin = [0,5,8]
category = pd.cut(wine.quality, bin, labels=[0, 1])
category = category.to_frame()
category.columns = ['qual_above5']
wine_ = pd.concat([wine.drop('quality', axis=1), category], axis=1)

In [13]:
wine_.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,qual_above5
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


Create a train test split with 20% of the data in the test subsample.

In [17]:
# answer below:
X = wine_.drop('qual_above5', axis=1)
Y = wine_.qual_above5

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Scale only the independent variables using the minmax scaler.

In [19]:
# answer below:
scale = MinMaxScaler()

X_train_ = scale.fit_transform(X_train)
X_test_ = scale.transform(X_test)

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [20]:
# answer below:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn_ = KNeighborsClassifier(n_neighbors=5)
knn_.fit(X_train_, y_train)

print(f'Non-scaled KNN accuracy: {knn.score(X_test, y_test)}')
print(f'Scaled KNN accuracy: {knn_.score(X_test_, y_test)}')

Non-scaled KNN accuracy: 0.625
Scaled KNN accuracy: 0.675


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [34]:
def _set_weight(dist):
    if dist.dtype is np.dtype(object):
        for point_dist_i, point_dist in enumerate(dist):
            if hasattr(point_dist, '__contains__') and 0. in point_dist:
                dist[point_dist_i] = point_dist == 0.
            else:
                dist[point_dist_i] = 1. / np.sqrt(point_dist)
    else:
        with np.errstate(divide='ignore'):
            dist = 1. / np.sqrt(dist)
        inf_mask = np.isinf(dist)
        inf_row = np.any(inf_mask, axis=1)
        dist[inf_row] = inf_mask[inf_row]
    return dist

In [35]:
n_weight = _set_weight(X_train_)

In [28]:
ideal_k = np.round(np.sqrt(len(wine_)))
print(f'Ideal K: {ideal_k}\n')

Ideal K: 40.0



In [ ]:
knn_d_ = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_d_.fit(X_train_, y_train)

In [ ]:
knn_c_ = KNeighborsClassifier(n_neighbors=5, weights=_set_weight)
knn_c_.fit(X_train_, y_train)

In [ ]:
knn_i_ = KNeighborsClassifier(n_neighbors=40, weights=_set_weight)
knn_i_.fit(X_train_, y_train)

In [ ]:
knn_if_ = KNeighborsClassifier(n_neighbors=40)
knn_if_.fit(X_train_, y_train)

In [58]:
print(f'\nNon-scaled KNN train accuracy: {knn.score(X_train, y_train)}')
print(f'Scaled KNN train accuracy: {knn_.score(X_train_, y_train)}')
print(f'Weighted(distance), Scaled KNN train accuracy: {knn_d_.score(X_train_, y_train)}')
print(f'Weighted(c), Scaled KNN train accuracy: {knn_c_.score(X_train_, y_train)}')
print(f'Ideal K, Weighted(c), Scaled KNN train accuracy: {knn_i_.score(X_train_, y_train)}')
print(f'Ideal K, Scaled KNN train accuracy: {knn_if_.score(X_train_, y_train)}')

print(f'\nNon-scaled KNN test accuracy: {knn.score(X_test, y_test)}')
print(f'Scaled KNN test accuracy: {knn_.score(X_test_, y_test)}')
print(f'Weighted(distance), Scaled KNN test accuracy: {knn_d_.score(X_test_, y_test)}')
print(f'Weighted(c), Scaled KNN test accuracy: {knn_c_.score(X_test_, y_test)}')
print(f'Ideal K, Weighted(c), Scaled KNN test accuracy: {knn_i_.score(X_test_, y_test)}')
print(f'Ideal K, Scaled KNN test accuracy: {knn_if_.score(X_test_, y_test)}')


Non-scaled KNN train accuracy: 0.7615324472243941
Scaled KNN train accuracy: 0.8248631743549648
Weighted(distance), Scaled KNN train accuracy: 1.0
Weighted(c), Scaled KNN train accuracy: 1.0
Ideal K, Weighted(c), Scaled KNN train accuracy: 1.0
Ideal K, Scaled KNN train accuracy: 0.7560594214229867

Non-scaled KNN test accuracy: 0.625
Scaled KNN test accuracy: 0.675
Weighted(distance), Scaled KNN test accuracy: 0.746875
Weighted(c), Scaled KNN test accuracy: 0.7375
Ideal K, Weighted(c), Scaled KNN test accuracy: 0.778125
Ideal K, Scaled KNN test accuracy: 0.721875


*With custom weights, the train accuracy becomes perfect which is initially  worrying. On the other hand, the best test scores comes from the model with an ideal K-value, custom weights, and scaled independent features.*